# The Woof Factor in Zürich
---

Through use of data freely available from *Stadt Zürich Open Data*, we analyse dogs registered in Zurich and combine this data with other information available about the districts (kreis) that make up Zurich. We perform exploratory analysis using a number of visualisations and a model is developed in which we predict dog breed given related data.

*Project Group 23:* Corey Bothwell, Nicoletta Farabullini, Jacob Gelling, Andris Prokofjevs, Qasim Warraich <br>
Link to Repo: https://github.com/nfarabullini/Big-Data-Analytics

# Configuration and Dependencies
---
## R Kernel
This Jupyter notebook requires use of the R Kernel, which can be installed with a very simple 2 step installation process.

The detailed instructions can be found here: https://irkernel.github.io/installation/

## Browser
We reccomend a **chromium based browser** to view this notebook. We have discovered an **issue** rendering one of the visualisations **in firefox**.

## Libraries
The required packages must first be installed. On Linux, this requires the system to have Curl (for communicating with the Yandex Translate API) and GDAL (for generating the map vizualisations) installed.

# R Packages
---


In [ ]:
install.packages("readr")
install.packages("data.table")
install.packages("RYandexTranslate")
install.packages("plyr")
install.packages("dplyr")
install.packages("stringr")
install.packages("class")
install.packages("e1071")
install.packages("ggplot2")
install.packages("reshape2")
install.packages("leaflet")
install.packages("rgdal")
install.packages("stringi")

Libraries used throughout the analysis are imported here.

In [ ]:
library(readr)
library(data.table)
library(RYandexTranslate)
library(plyr)
library(dplyr)
library(stringr)
library(class)
library(e1071)
library(ggplot2)
library(reshape2)
library(leaflet)
library(rgdal)
library(stringi)

# Data Aquisition
---

All data was aquired from Stadt Zürich Open Data. The below table details data sources used and their respective URL. This data is processed, cleaned and merged in the next section.

| Content           | URL                                                                                      |
|-------------------|------------------------------------------------------------------------------------------|
| Dog Registrations | https://data.stadt-zuerich.ch/dataset/sid_stapo_hundebestand                             |
| Wealth Data       | https://data.stadt-zuerich.ch/dataset/fd_median_einkommen_quartier_od1003                |
| Income Data       | https://data.stadt-zuerich.ch/dataset/fd_median_vermoegen_quartier_od1004                |
| Education Data    | https://data.stadt-zuerich.ch/dataset/bfs_bev_bildungsstand_statquartier_seit1970_od1012 |
| Home Type Data    | https://data.stadt-zuerich.ch/dataset/bau_best_geb_whg_bev_gebaeudeart_quartier_seit2008 |

# Data Processing and Cleaning
---

This section is used for importing the raw data from CSV files into R data tables. Appropriate data merging and cleaning is performed.

Also performed is automatic translations from German to English for some of the column content that would otherwise be too laborious to do by hand.

Rather than running data cleaning each time we run the notebook during development, which can be slow due to the Yandex Translate API, an R data image of the alreadly cleaned data provided with the Notebook file can be loaded instead by running the following block.

In [ ]:
load("dogs.Rdata")

## Import Dog Data

The dog registration data for 2020, for which the source is available at [Stadt Zürich Open Data](https://data.stadt-zuerich.ch/dataset/sid_stapo_hundebestand/resource/a05e2101-7997-4bb5-bed8-c5a61cfffdcf), is imported and converted to a R data table.

In [ ]:
dogs2020 <- data.table(read_csv("data_sources/20200306_hundehalter.csv"))

Data cleaning is now performed. Due to what is likely a typographical error, a dog with a district value of 8 is removed (non-existant stadtquartier).

Data we are uninterested in and rows that are incomplete are also removed, however for the 2020 dog dataset we find that 0 rows are ommited.

In [ ]:
# Dog w. district "8", typographical error? We remove it.
dogs2020 <- dogs2020[(dogs2020$STADTQUARTIER!=8), ]

# Removing unnececesary columns
dogs2020[, c("RASSE1_MISCHLING", "RASSE2", "RASSE2_MISCHLING"):=NULL]

# If a row has a NA entry in one of the cells, remove the entire row
dogs2020 <- na.omit(dogs2020)
# 0 rows ommited, still leave the code in place in case we change data basis.

Due to the international makeup of our team members, we translate the column names from German to English.

In [ ]:
# Rename columns
setnames(dogs2020,
         old = c("HALTER_ID", "ALTER", "GESCHLECHT", "STADTQUARTIER", "STADTKREIS",   "RASSE1", "RASSENTYP",  "GEBURTSJAHR_HUND", "GESCHLECHT_HUND", "HUNDEFARBE"),
         new = c("OWNER_ID",  "AGE",   "SEX",        "DISTRICT",      "DISTRICT_BIG", "BREED",  "BREED_TYPE", "YOB_DOG",          "SEX_DOG",         "COLOR_DOG")
        )

Because stadtquartier is a more granular district segmentation than stadtkreis, and not all datasets can be merged by stadtkreis, we used it instead. We found that it is hard to view on a map when the dataset does not contain normal district names, so we extract the district names from the wealth dataset.

In [ ]:
# Import district names
district_names <- data.table(read_csv("data_sources/wir100od1004.csv"))
district_names <- unique(district_names[, QuarSort, QuarLang])

# Rename columns
setnames(district_names,
         old = c("QuarSort", "QuarLang"),
         new = c("DISTRICT", "DISTRICT_NAME")
        )

dogs2020 <- merge(dogs2020, district_names , by = "DISTRICT", all.x = T)

# Remove unused variable
rm(district_names)

## Import Wealth Data

Next the wealth data, for which the source is available at [Stadt Zürich Open Data](https://data.stadt-zuerich.ch/dataset/fd_median_einkommen_quartier_od1003), is imported and converted to a R data table.

In [ ]:
wealth <- data.table(read_csv("data_sources/wir100od1004.csv"))

The latest data from this source is for 2017, so we select only this wealth data, store the average and aggregate the data.

In [ ]:
# We don't have data for 2020, the freshest data is for 2017
wealth <- wealth[SteuerJahr == 2017,]

# Create a new column to store the average
wealth[, wealth50 := SteuerVermoegen_p50]

# Replace the family values in this column with same divided by 2 for normalization
wealth$wealth50[wealth$SteuerTarifSort == 1] <- wealth$wealth50[wealth$SteuerTarifSort == 1]/2

# Aggregate data for family status
# new table = old table, select mean of wealth50 (ignore NA), aggregate it by quartal
wealth_merge <- wealth[,mean(wealth50, na.rm = T), by=QuarSort]

# Leaving only quartals that have dogs in them
wealth_merge <- wealth_merge[wealth_merge$QuarSort %in% dogs2020$DISTRICT]

We again translate the column names from German to English and then merge the data with the dogs data table.

In [ ]:
# Rename columns
setnames(wealth_merge,
         old = c("V1",           "QuarSort"),
         new = c("WEALTH_T_CHF", "DISTRICT")
        )

# Merge wealth into dogs dataset
dogs2020 <- merge(dogs2020, wealth_merge, by = "DISTRICT", all.x = T)
# ATTENTION! FYI: not all districts have wealth values! 

# Remove unused variables
rm(wealth, wealth_merge)

## Import Income Data

Next the income data, for which the source is available at [Stadt Zürich Open Data](https://data.stadt-zuerich.ch/dataset/fd_median_vermoegen_quartier_od1004), is imported and converted to a R data table.

In [ ]:
income <- data.table(read_csv("data_sources/wir100od1003.csv"))

Again, the latest data from this source is for 2017, so we select only this income data, store the average and aggregate the data.

In [ ]:
# We don't have data for 2020, the freshest data is for 2017
income <- income[SteuerJahr == 2017,]

# Create a new column to store the average
income[, incomep50 := SteuerEInkommen_p50]

# Replace the family values in this column with same divided by 2 for normalization
income$incomep50[income$SteuerTarifSort == 1] <- income$incomep50[income$SteuerTarifSort == 1]/2

# Aggregate data for family status
# new table = old table, select mean of incomep50 (ignore NA), aggregate it by quartal
income_merge <- income[,mean(incomep50, na.rm = T), by=QuarSort]

# Leaving only quartals that have dogs in them
income_merge <- income_merge[income_merge$QuarSort %in% dogs2020$DISTRICT]

We again translate the column names from German to English and then merge the data with the dogs data table.

In [ ]:
# Rename columns
setnames(income_merge,
         old = c("V1",           "QuarSort"),
         new = c("INCOME_T_CHF", "DISTRICT")
        )

# Merge income into dogs dataset
dogs2020 <- merge(dogs2020, income_merge, by = "DISTRICT", all.x = T)
# ATTENTION! FYI: not all districts have income values! 

# Remove unused variables
rm(income, income_merge)

## Import Education Data

The import of education data is straightforward, requiring only a long to wide reshape of the data. The data source is available at [Stadt Zürich Open Data](https://data.stadt-zuerich.ch/dataset/bfs_bev_bildungsstand_statquartier_seit1970_od1012).

In [ ]:
education <- data.table(read_csv("data_sources/bil101od1012 (2).csv"))

# Long to wide education reshape
education <- dcast(education, RaumSort ~ Bildungsstand, value.var = "AntBev")

# Rename columns
setnames(education,
         old = c("RaumSort", "Obligatorische Schule",   "Sekundarstufe II",     "Tertiärstufe"),
         new = c("DISTRICT", "BASIC_SCHOOL_PERCENTAGE", "GYMNASIUM_PERCENTAGE", "UNIVERSITY_PERCENTAGE")
        )

# Merge income into dogs dataset
dogs2020 <- merge(dogs2020, education, by = "DISTRICT", all.x = T)

# Remove unused variable
rm(education)

## Import Home Type Data

Home type data, such as number of resident buildings in a district, is now imported. The data source is available at [Stadt Zürich Open Data](https://data.stadt-zuerich.ch/dataset/bau_best_geb_whg_bev_gebaeudeart_quartier_seit2008/resource/3850add1-264c-4993-98cd-d8a9ba87ee25). 2019 is the latest data available. Each building type is summed and a long to wide reshape of the data is performed.

In [ ]:
home_type <- data.table(read_csv("data_sources/bau_best_geb_whg_bev_gebaeudeart_quartier_seit2008.csv")) 

# We don't have data for 2020, the freshest data is for 2019
home_type <- home_type[Jahr == 2019,]

# Count the sum of every type of building
home_type <- home_type[, sum(AnzGeb), by = list(QuarSort,GbdArtPubName)]

# Rename columns
setnames(home_type,
         old = c("QuarSort", "GbdArtPubName", "V1"), 
         new = c("DISTRICT", "Hometype",      "Number_homes")
        )

# Long to wide hometype reshape
home_type <- data.table::dcast(home_type, DISTRICT ~ Hometype, value.var = "Number_homes")

# Translate hometype columns
setnames(home_type,
         old = c("Produktions- und Lagergebäude", "Mehrfamilienhäuser", "Einfamilienhäuser",   "Infrastrukturgebäude",     "Kleingebäude",    "Kommerzielle Gebäude", "Spezielle Wohngebäude"), 
         new = c("FACTORIES_AND_WAREHOUSES",      "APARTMENTS",         "SINGLE_FAMILY_HOMES", "INFRASTRUCTURE_BUILDINGS", "SMALL_BUILDINGS", "COMMERCIAL_BUILDINGS", "SPECIAL_ACCOMODATION")
        )

# Remove unnececesary column
home_type[, Unbekannt:=NULL]

# Merge home types into dogs dataset
dogs2020 <- merge(dogs2020, home_type, by = "DISTRICT", all.x = T)

# Remove unused variable
rm(home_type)

## Import Population Data

Next population data is imported. The data source is available at [Stadt Zürich Open Data](https://www.stadt-zuerich.ch/prd/de/index/statistik/themen/bevoelkerung/bevoelkerungsentwicklung/kreise-und-quartiere.html#daten). The source CSV file is a little messy, so some additional data cleaning is required. When importing, the first 8 lines are skipped for the parser to reach the CSV header. As one of the columns are unlabelled, a warning is produced.

In [ ]:
# Warning shown due to formatting of souce CSV header, can be ignored
pop_per_district <- data.table(read_csv("data_sources/2019-Table_1.csv", col_names = TRUE, skip = 8))

# Set column names
setnames(pop_per_district, old = "X1", new = "DISTRICT_NAME")

The data also contains hidden characters acting as whitespace which the standard regex matching pattern ```\\s``` does not always catch. For this reason we remove all non-numeric characters with a regex function.

In [ ]:
# This Reg-Ex Matching Function Removes all Whitespaces and not only conventional ones (bad data design)
pop_per_district[,2:5] <- data.table(apply(pop_per_district[,2:5], 2, function(x) gsub('[^0-9.]', '', x)))

As normal, we continue by merging the population data into the dogs data.

In [ ]:
# Join by district name (perfect match with no NAs)
dogs2020 <- merge(dogs2020, pop_per_district, by = "DISTRICT_NAME", all.x = T)

# Remove unused variable
rm(pop_per_district)

# Rename columns
setnames(dogs2020,
         old = c("Total",            "Schweizer/-innen", "Ausländer/-innen",   "Anteil ausländische\nBevölkerung (%)"),
         new = c("TOTAL_POPULATION", "SWISS_POPULATION", "FOREIGN_POPULATION", "FOREIGN_POPULATION_PERCENTAGE")
)

The columns are then cast as numeric data types for use in mathematical operations later in the analysis.

In [ ]:
# Cast columns as numeric
dogs2020$TOTAL_POPULATION <- as.numeric(dogs2020$TOTAL_POPULATION)
dogs2020$SWISS_POPULATION   <- as.numeric(dogs2020$SWISS_POPULATION)
dogs2020$FOREIGN_POPULATION   <- as.numeric(dogs2020$FOREIGN_POPULATION  )
dogs2020$FOREIGN_POPULATION_PERCENTAGE   <- as.numeric(dogs2020$FOREIGN_POPULATION_PERCENTAGE)

## Translate Dataframes
Using the Yandex Translate API, values in columns are automatically translated from German to English.

Firstly the Yandex translate package is fixed, as per the [solution found on GitHub](https://github.com/mukul13/RYandexTranslate/issues/2).

In [ ]:
translate = function (api_key, text = "", lang = "") 
{
  url = "https://translate.yandex.net/api/v1.5/tr.json/translate?"
  url = paste(url, "key=", api_key, sep = "")
  if (text != "") {
    url = paste(url, "&text=", text, sep = "")
  }
  if (lang != "") {
    url = paste(url, "&lang=", lang, sep = "")
  }
  url = gsub(pattern = " ", replacement = "%20", x = url)
  d = RCurl::getURL(url, ssl.verifyhost = 0L, ssl.verifypeer = 0L, .encoding = "UTF-8")
  d = jsonlite::fromJSON(d)
  d$code = NULL # Remove unused variables
  d
}

As the Yandex Translate API requires an API key, we define this here. It is connected to Andris' personal account, so please do not disclose it elsewhere.

In [ ]:
# Andris' personal Yandex Translate API key
# PLEASE DO NOT DISCLOSE ELSEWHERE
api_key <- "trnsl.1.1.20200515T134653Z.f9fb709ac3e94036.783aefa609692b463a79b5827d5c0e7f2d037a8c"

Next we define the list of columns for which we want to translate their contents.

In [ ]:
column_list <- c("BREED", "COLOR_DOG")

We then loop through columns that we wish to translate and pass these to the Yandex Translate API. As translation can be slow, we pass only unique values from the columns, preventing a translation call for every row, even if the value was translated before. The output is saved to the dogs data table.

In [ ]:
# Loop through columns in column_list
for (column_names in column_list) {
  # Get all unique values in the coresponding column
  unique_values <- unique(dogs2020[, get(column_names)])
    
  # replacing special character (apostrophy) in one of the breed names, to avoid encoding error in jupyter
  unique_values <- str_replace(unique_values, "\032" , "")
    
  # Loop through unique values to be translated
  for (unique_num in 1:length(unique_values)) {
    
    # Print progress for debugging
    print(column_names)
    print(paste(unique_num, " out of ", length(unique_values)))
      
    # data.table synthaxis code
    dogs2020[
      # Select all rows where value is equal to current unique value
      # left part "get" is needed in order to use dynamic column_names
      dogs2020[,get(column_names)] == unique_values[unique_num],
      # Replace old value with translated one
      # Left part is "eval" is needed in order to use dynamic column_names
      # Right part is a Yandex Translate function, providing api_key and text to be translated
      eval(column_names) := translate(api_key, text=stringi::stri_unescape_unicode(unique_values[unique_num]),
      # Specify language pair and translation direction
      lang="de-en"
      # Extract only output text from the return given by Yandex
      )$text]
  
  }
}

We manually replace sex and sex_dog variable to match the English language conventions for sex abbreviations.

In [ ]:
dogs2020[SEX == "w", SEX := "f"]
dogs2020[SEX_DOG == "w", SEX_DOG := "f"]

## Save RData Image
An R data image is saved here so that instead of running the entire data cleaning code block again in future runs, the image can be loaded.

In [ ]:
save.image("dogs.Rdata")

---
## Column Overview

**DISTRICT_NAME** = String representation of the district's name <br>
**DISTRICT** = Numeric representation of a sub district <br>
**OWNER_ID** = Unique owner ID <br>
**AGE** = Age group of owner (Grouped by decade)  <br>
**SEX** = Gender of owner  <br>
**DISTRICT_BIG** = Parent of subdistrict  <br>
**BREED** = String representation of breed of dog  <br>
**BREED_TYPE** = 3 possible values (K,I,II) K = Small Breeds, I = Unrestricted Breeds, II = Restricted Breeds.<br>
**YOB_DOG** = Year of dog's birth <br>
**SEX_DOG** = Gender of Dog (M or F) <br>
**COLOR_DOG** = String representation of dog colour <br>
**WEALTH_T_CHF** = Avg district resident wealth in thousands of CHF. <br>
**INCOME_T_CHF** = Avg district resident income in thousands of CHF. <br>
**BASIC_SCHOOL_PERCENTAGE** = Avg district resident's who have <= Basic school education. <br>
**GYMNASIUM_PERCENTAGE** = Avg district resident's who have <= Gymnasium level education. <br>
**UNIVERSITY_PERCENTAGE** = Avg district resident's who have <= University level education. <br>
**SINGLE_FAMILY_HOMES** = Number of single family homes in a given sub-district.  <br>
**INFRASTRUCTURE_BUILDINGS** = Number of infrastructure buildings in a given sub-district. <br>
**SMALL_BUILDINGS** = Number of small buildings in a given sub-district.  <br>
**COMMERCIAL_BUILDINGS** = Number of commercial buildings in a given sub-district.<br>
**APARTMENTS** = Number of apartment buildings in a given sub-district.<br>
**FACTORIES_AND_WAREHOUSES** = Number of factories and warehouses in a given sub-district.<br>
**SPECIAL_ACCOMODATION** = Number of special accomodations (eg: dorms, carehomes) in a given sub-district.<br>
**TOTAL_POPULATION** = Total population of a given sub-district.<br>
**SWISS_POPULATION** = Total Swiss population of a given sub-district. <br>
**FOREIGN_POPULATION** = Total foreign population of a given sub-district.<br>
**FOREIGN_POPULATION_PERCENTAGE** = Total foreign population percentage of a given sub-district.<br>

## Limitations

It is important to discuss some limitations inherent in the data. To wit, the import data often cannot be matched up exactly year to year. Concretely, sources in our data imports maintain years between 2017 to 2020. This could skew the results in an inaccurate manner if large variations in the measured data (for instance, wealth) were present in the unmeasured years. A critical assumption in our analysis is therefore that these variables do not vary drastically from year to year, and that an appropriate 'general' model can be inferred with years that do not match one-to-one.

Futhermore, within our income & wealth datasets, some districts do not maintain a value across one or more of the three categorical dimensions (family type) of the appropriate wealth or income class. To account for this, we have computed an average across all available categorical dimensions. If these missing values, which function as omitted variables, were to significantly alter the underlying distribution, our results may not hold in practice.

Finally, it is important to note that the data is effectively split into two sets of granularities. The first are those characteristics of individual dogs and owners themselves, (Sex of Dog, Age of Owner, etc.) while the second is are those characteristics computed across districts in aggregate. These include features such as Average Income, Average Wealth, etc. and those features can be utilised as proxies on individual owners in those districts. While less severe to underlying stastistical distributions than the other limitations mentioned in this paragraph, it behooves practicioners to keep this distinction in granularity in mind when conducting analysis. We account for this important distinction by effectively splitting our analysis into two classes, that which applies to owners & dogs individually, and that which applies to districts in aggregate.

## Backup Output
Save a backup of the dogs data set in another variable. This is useful as the data is modified during the investigation. **This backup is not optional, please execute this block below.**

In [ ]:
dogs2020_backup = copy(dogs2020)

# Summary Statistics

Finally, before we continue, let us compute some basic summary statistics from the dataset that will give us an overview of the distribution of dogs and their owners in Zuerich and that will help us later on.



In [ ]:
unique_gender <- unique(dogs2020$SEX)
total_gender <- lapply(unique_gender, function(x){length(which(dogs2020$SEX == x))})
percent_gender <- round(unlist(total_gender)/sum(unlist(total_gender))*100)

unique_age <- unique(dogs2020$AGE)
total_age <- lapply(unique_age, function(x){length(which(dogs2020$AGE == x))})
percent_age <- round(unlist(total_age)/sum(unlist(total_age))*100)

# Table illustrates the percentage of males and females owning a dog
table_gender <- data.table("Owner Gender" = unique_gender, "Percentage %" = percent_gender)

# Table illustrates the percentage of dog ownership for each age group
table_age <- data.table("Owner Age Group" = unique_age, "Percentage %" = percent_age)
setorder(table_age, "Owner Age Group")

Let's look at those tables:

In [ ]:
table_gender
table_age

Wow! So Females own dogs in a much higher amount than males for our sample. (population of Zuerich) This will play an important role in our analysis later on. 


# Visualisation

### Maps and Charts
In order to effectively perform some exploratory data analysis, we decide to build a visualisation over the districts of Zürich. We are interested in providing a mental map for how wealth is distributed across the city. A lot of the analysis presented in this notebook hinges on economic analysis so we thought this would be a fitting visualisation to accompany the rest of the material present in the notebook.

#### Here we group the subdistricts of Zurich together.

In [ ]:
zh_rg <- readOGR("./data_sources/stzh.adm_stadtkreise_v.json")

# group sub-districts together
list_districts <- list(
  District_1 <- c(
    "Rathaus",
    "Hochschulen",
    "Lindenhof",
    "City"
  ),
  District_2	<- c(
    "Wollishofen",
    "Leimbach",
    "Enge"
  ),
  District_3 <- c(
    "Alt-Wiedikon",
    "Friesenberg",
    "Sihlfeld"
  ),
  District_4 <- c(
    "Werd",
    "Langstrasse",
    "Hard"
  ),
  District_5 <- c(
    "Gewerbeschule",
    "Escher Wyss"
  ),
  District_6	<- c(
    "Unterstrass",
    "Oberstrass",
    "Unterstrass"
  ),
  District_7 <- c(		
    "Fluntern",
    "Hottingen",
    "Hirslanden",
    "Witikon"
  ),
  District_8 <- c(
    "Seefeld",
    "Mühlebach",
    "Weinegg"
  ),
  District_9 <- c(	
    "Albisrieden",
    "Altstetten"
  ),
  District_10 <- c(	
    "Höngg",
    "Wipkingen"
  ),
  District_11 <- c(	
    "Affoltern",
    "Oerlikon",
    "Seebach"
  ),
  District_12 <- c(
    "Saatlen",
    "Schwamendingen-Mitte",
    "Hirzenbach"
  )
)


#### We then calculate the average wealth per city districts.

In [ ]:
avg_wealth <- unlist(lapply(seq_len(length(list_districts)), function (z) {
  mean(unlist(lapply(list_districts[[z]], function (y) {
    mean(unlist(lapply(y, function (x) {
      dogs2020[which(x == dogs2020$DISTRICT_NAME),]$WEALTH_T_CHF
    })))
  })))
}))

#### This block handles the colouring and generation of a leaflet map that will show a district wise wealth map of Zürich.

In [ ]:
bins <- c(0, 25, 30, 50, 60, 75, 100, 120, 150, 175)
pal <- colorBin("Greens", domain = avg_wealth, bins = bins)

map <- leaflet(zh_rg) %>%
  addPolygons(fillColor = ~pal(unlist(avg_wealth)), weight = 2, fillOpacity = 0.9, 
              opacity = 1) %>%
  addTiles() %>% 
  addLegend(colors = pal(unlist(avg_wealth)), labels = zh_rg$kname, title = "Zurich Districts", opacity = 1)


## Leaflet map
This map shows a district wise breakdown of wealth in the City of Zürich. The darker colours represent lower wealth where as the lighter shades represent wealthier districts. In other words, Kreis 12 is the poorest district, whereas Districts 1 and 2 are the wealthiest. <br>
**Note: You may need to execute the block below twice to actually get the map to appear.** 

In [ ]:
map

# Modelling and Analysis
---
This section of code explores predicting dog breed, starting from a sophisticated KNN Algorithm (both a naive approach and moving onwards) and then moves on to a more traditional Naive Bayes approach.

## K-Nearest Neighbours Classification

Our problem is a classification problem because we have labled data and our goal is to predict a class of new data based on known attributes. There are multiple appropriate classification algorithms for such a task, we arbritraily chose KNN to serve as benchmark for the dataset to see what weaknesses we could identify in this model. This was in an attempt to narrow down on a more appropriate model for our data analysis. 

### Model Preparation

In [ ]:
dogs2020$COLOR_DOG[1:20]

First we recode color to make bigger chunks. **Note: Clearing variable i from the enviroment can produce a warning on first-run of the code, however is necessary on subsequent runs and the error can be safely ignored.** As you can see some dogs are multicoloured so too many distinct colour groups that do not represent the differences between the dogs are formed. To combat this we reduce the colours to only the first listed or primary colour of the dog. 


In [ ]:
# Double-colored dogs will be recoded as single-colored with first color taken as main
a <- str_split(dogs2020$COLOR_DOG, "/")
new_colors <- a[[1]][1]
rm(i)
for (i in 2:length(a)) {
  new_colors <- append(new_colors, a[[i]][1])
}

# Put new colors into the dogs data table
dogs2020$COLOR_DOG <- new_colors

As we are going to be putting the data into a knn model, we must first refactor the data into numeric values. This is done for all values we are interested in. The associated labels are saved for breed and breed type.

In [ ]:
# Refactoring text attributes to numeric, so that knn can work with it
dogs2020$DISTRICT_NAME <- as.numeric(as.factor(dogs2020$DISTRICT_NAME))
dogs2020$AGE <- as.numeric(as.factor(dogs2020$AGE))
dogs2020$SEX <- as.numeric(as.factor(dogs2020$SEX))
dogs2020$SEX_DOG <- as.numeric(as.factor(dogs2020$SEX_DOG))
dogs2020$COLOR_DOG <- as.numeric(as.factor(dogs2020$COLOR_DOG))
dogs2020$YOB_DOG <- as.numeric(as.factor(dogs2020$YOB_DOG))

# Saving labels in order to see later what numbers represent what breedtypes
breedtype_labels <- as.factor(dogs2020$BREED_TYPE)
dogs2020$BREED_TYPE <- as.numeric(as.factor(dogs2020$BREED_TYPE))

# Saving labels in order to see later what numbers represent what breeds
breed_labels <- as.factor(dogs2020$BREED)
dogs2020$BREED <- as.numeric(as.factor(dogs2020$BREED))

In order to train and test the data, we create training (70%) and testing (30%) datasets.

In [ ]:
# Data separation, so that we have testing and training group
set.seed(1) # We set this to ensure reproducibility and result consistencey.


dat.d <- sample(1:nrow(dogs2020),size=nrow(dogs2020)*0.7,replace = FALSE) # Randomly select 70% of the data.

# This is the only setting where we are not "cheating" by using dog associated attributes
train.dogs <- dogs2020[dat.d, c("DISTRICT_NAME", "AGE", "SEX")] # 70% training data
test.dogs <- dogs2020[-dat.d, c("DISTRICT_NAME", "AGE", "SEX")] # 30% testing data

# Provide labels for training and testing datasets
train.dogs_labels <- dogs2020[dat.d,BREED]
test.dogs_labels <-dogs2020[-dat.d,BREED]

## "Naive" KNN
Here we perform a simple k-nearest neighbors classification. We try to directly predict breeds by providing all available parameters to the model (excluding population and economic data because they are district level variables and not granular enough to be used on a per breed basis). The accuracy of this direct prediction is roughly 9.22%.

In the table below you can see the output of the KNN model prediction for breed vs the real breed values.

In [ ]:
# Calculate an approx. k value
k_value <-  sqrt(nrow(dogs2020))

# Perform knn classification using given training data and testing data
knn.test <- knn(train=train.dogs, test=test.dogs, cl=train.dogs_labels, k=k_value)

# Calculate accuracy
ACC.test <- 100 * sum(test.dogs_labels == knn.test) / NROW(test.dogs_labels)

# Print results
table(knn.test ,test.dogs_labels)

#### Accuracy test output.

In [ ]:
ACC.test 

## KNN with Breed Type Grouping (Increasing Model Accuracy)

Based on the relatively low accuracy of ~9% we next decided to try and further separate the dataset into smaller chunks with the idea that it would decrease chunk level variety and potentially increase accuracy. As a chunk seperator we used breed types because it's a natural seperator that existed in our dataset. 

In [ ]:
# This is the point where we exclude all population and income level data due to the granularity issues discussed 
# above.

train.dogs <- dogs2020[dat.d, c("DISTRICT_NAME", "AGE", "SEX")] # 70% training data
test.dogs <- dogs2020[-dat.d,c("DISTRICT_NAME", "AGE", "SEX")] # Remaining 30% test data

train.dogs_labels <- dogs2020[dat.d,BREED_TYPE]
test.dogs_labels <-dogs2020[-dat.d,BREED_TYPE]

#Approximate k value
k_value <-  sqrt(nrow(dogs2020))

knn.test <- knn(train=train.dogs, test=test.dogs, cl=train.dogs_labels, k=k_value)

ACC.test <- 100 * sum(test.dogs_labels == knn.test)/NROW(test.dogs_labels)

table(knn.test ,test.dogs_labels)

#### Output of accuracy test

In [ ]:
ACC.test

#### This apporach yields ~62% accuracy in breed type prediction now we want to predict exact breed in order to do so we need to save breed type prediction. 

In [ ]:
# we save predicted breedtype, by applying algorithm to whole dataset. (this can open potential for overfitting)
train.dogs$PREDICTED_BREED_TYPE <- knn(train=train.dogs, test=train.dogs, cl=train.dogs_labels, k=k_value)
test.dogs$PREDICTED_BREED_TYPE <- knn(train=train.dogs[,c("DISTRICT_NAME", "AGE", "SEX")], test=test.dogs, cl=train.dogs_labels, k=k_value)

#### Perform KNN on breed directly


In [ ]:
rm(i) # Safety measures
results <- 0 # Safety measures
for (i in 1:length(unique(test.dogs_labels))) # Loops through all unique dog breed types.
{ 
  if (any(train.dogs$PREDICTED_BREED_TYPE == i) == F) {  
    next()
  }
  
  train.dogs_labels <- dogs2020[dat.d,BREED] # Label assignment for testing and training datasets,
  test.dogs_labels <- dogs2020[-dat.d,BREED] # because it was previously used for breed types not breeds.
  
  # Calculating the model per breed type.   
  knn.test <- knn(train=train.dogs[train.dogs$PREDICTED_BREED_TYPE == i], 
                  test=test.dogs[test.dogs$PREDICTED_BREED_TYPE == i], 
                  cl=train.dogs_labels[train.dogs$PREDICTED_BREED_TYPE == i], k=k_value)
  
  ACC.test <- 100 *
    sum(test.dogs_labels[test.dogs$PREDICTED_BREED_TYPE == i] == knn.test)/
    NROW(test.dogs_labels[test.dogs$PREDICTED_BREED_TYPE == i])
  
  table(knn.test ,test.dogs_labels[test.dogs$PREDICTED_BREED_TYPE == i])
  
  results[i] <- ACC.test
}

#### Calculating weighted accuracy:

In [ ]:
(length(test.dogs_labels[test.dogs$PREDICTED_BREED_TYPE == 1]) * results[1]/100
  + length(test.dogs_labels[test.dogs$PREDICTED_BREED_TYPE == 3]) * results[3]/100 ) / 
  (length(test.dogs_labels[test.dogs$PREDICTED_BREED_TYPE == 1]) + length(test.dogs_labels[test.dogs$PREDICTED_BREED_TYPE == 3]))*100

#### The result of ~9%, is not substantially better than approach without breed type grouping.

---
## "Naive" KNN with Breed Frequency Filtering.
As a second attempt to improve accuracy we remove certain breeds that are not highly represented in the data in order to reduce the possible number of outcomes. This approach does not compromise the strength of the prediction as certain breeds appear an insufficient (< 20) amount of times in the dataset.


##### In order to test it, we have to calculate how many people own each breed:


In [ ]:
breed_filter <- data.table(aggregate(OWNER_ID ~ BREED, data = dogs2020, FUN = function(x){NROW(x)}))
# Renaming
setnames(breed_filter, old = c("OWNER_ID")
         , new = c("NUMBER_OWNERS"))

# Sorting by number of owners
breed_filter <- breed_filter[order(-rank(NUMBER_OWNERS),)]
breed_filter <- breed_filter[breed_filter[, NUMBER_OWNERS > 20]]

# Leaving in dataset only desired breeds
dogs2020 <- dogs2020[BREED %in% breed_filter[, BREED]]

# 1342 out of 7839 rows are out. (6497 left)
# 64 breeds out of 313 breeds left

#### Recalculation of the naive approach but with the filtering of breeds described above:

In [ ]:

# Now we rerun an exact copy of code in the first try

dat.d <- sample(1:nrow(dogs2020),size=nrow(dogs2020)*0.7,replace = FALSE) #random selection of 70% data.

# This is the only setting where we are not "cheating" by using dog associated attributes
train.dogs <- dogs2020[dat.d, c("DISTRICT_NAME", "AGE", "SEX")] # 70% training data
test.dogs <- dogs2020[-dat.d,c("DISTRICT_NAME", "AGE", "SEX")] # Remaining 30% test data

train.dogs_labels <- dogs2020[dat.d,BREED]
test.dogs_labels <-dogs2020[-dat.d,BREED]

#Aprox k value
k_value <-  sqrt(nrow(dogs2020))

knn.test <- knn(train=train.dogs, test=test.dogs, cl=train.dogs_labels, k=k_value)

ACC.test <- 100 * sum(test.dogs_labels == knn.test)/NROW(test.dogs_labels)

table(knn.test ,test.dogs_labels)

#### Result of accuracy test

In [ ]:
ACC.test

 #### Accuracy of direct prediction is ~11%. Result is better, but still not significantly improven.

---
## KNN with  Frequency Based Grouping

Based on the previous KNN data we go on to gorup breeds with similar frequencies into groups inorder to produce more chunks with breeds that are more comparable in terms of frequency to avoid potential overpreferencing with KNN. It is a similar idea to the above Breed Type based grouping but with a different form. 


In [ ]:
# Calculating values for grouping
owners_sum <- sum(breed_filter$NUMBER_OWNERS)
number_groups <- 6
groupsize <- round(owners_sum / number_groups)

# Adding grouping variable
breed_filter[,group:=0]

# In any situation first value is group 1
breed_filter$group[1] <- 1

# Assigning groups
for (i in 2:nrow(breed_filter)) {
  if (sum(breed_filter[group == breed_filter$group[i-1], NUMBER_OWNERS]) > groupsize){
    breed_filter$group[i] <- breed_filter$group[i-1]+1
  } else {
    breed_filter$group[i] <- breed_filter$group[i-1]
  }
}
breed_filter

#### Preperation for KNN and Execution of KNN

In [ ]:
# Transfer grouping to original dataset
dogs2020 <- merge(dogs2020, breed_filter[,c("BREED", "group")], by = "BREED", all.x = T)

# Rewrite train and test datsets.
# This is the only setting where we are not "cheating" by using dog associated attributes
train.dogs <- dogs2020[dat.d, c("DISTRICT_NAME", "AGE", "SEX", "group")] # 70% training data
test.dogs <- dogs2020[-dat.d,c("DISTRICT_NAME", "AGE", "SEX","group")] # remaining 30% test data

train.dogs_labels <- dogs2020[dat.d,BREED]
test.dogs_labels <-dogs2020[-dat.d,BREED]

 ####  Here we once more execute code very similar to the above part with breed_type

In [ ]:
rm(i)
results <- 0
for (i in 1:length(unique(breed_filter$group)))
{
  if (any(train.dogs$group == i) == F) {
    next()
  }
  
  train.dogs_labels <- dogs2020[dat.d,BREED]
  test.dogs_labels <- dogs2020[-dat.d,BREED]
  
  knn.test <- knn(train=train.dogs[train.dogs$group == i], 
                  test=test.dogs[test.dogs$group == i], 
                  cl=train.dogs_labels[train.dogs$group == i], k=k_value)
  
  ACC.test <- 100 *
    sum(test.dogs_labels[test.dogs$group == i] == knn.test)/
    NROW(test.dogs_labels[test.dogs$group == i])
  
  table(knn.test ,test.dogs_labels[test.dogs$group == i])
  
  results[i] <- ACC.test
}

#### Calculating weighted accuracy:

In [ ]:
(length(test.dogs_labels[test.dogs$group == 1]) * results[1]/100 
  + length(test.dogs_labels[test.dogs$group == 2]) * results[2]/100
  + length(test.dogs_labels[test.dogs$group == 3]) * results[3]/100
  + length(test.dogs_labels[test.dogs$group == 4]) * results[4]/100
  + length(test.dogs_labels[test.dogs$group == 5]) * results[5]/100
  + length(test.dogs_labels[test.dogs$group == 6]) * results[6]/100) / 
    (length(test.dogs_labels[test.dogs$group == 1]) 
  + length(test.dogs_labels[test.dogs$group == 2])
  + length(test.dogs_labels[test.dogs$group == 3])
  + length(test.dogs_labels[test.dogs$group == 4])
  + length(test.dogs_labels[test.dogs$group == 5])
  + length(test.dogs_labels[test.dogs$group == 6])) * 100

The accuracy is ~29%, which is substantially better than 9% we got in the beginning. This is the best result we could get. However, the knn basically still follows the distribution of data. Because of that, one might consider invoking Occam's Razor, that is to say, perhaps a simpler more robust approach might lead to a comparable result with less complexity.

## Restore Backup
(Delete later once investigation does not directly edit dogs2020)

In [ ]:
dogs2020 = copy(dogs2020_backup)

---
# A Second Approach - Naive Bayes

##  Who's the master anyway?
Faced with some of the considerations of the KNN model, let's revisit our problem with a different approach.

In this section we start again from square one with our basic question: Do any characteristics of Dog Owners influence their choice of Dog Breed, and if so, which ones?

Using a Naive Bayes model we calculate frequency values for unique dog breeds against certain charateristics of the owners, specifically, age and gender. We also plot these frequency calculations on a per breed basis to better visualise the output of the model.



### Subset Data
First we subset the main dataset into those specific characteristics of individual dogs and owners.
We then remove less common breeds in order to present a tractable result.

It is very important to note that this is the same numerical approach utilised while performing the 3rd KNN model above (namely Naive KNN with Breed Freq. Filtering). In doing so we illuminate some of the advantages and tradeoffs inherent in model selection, especially the necessary consideration that must be given between more sophisticated with inherent fragility and more traditional models.

Finally we declare a helper variable of these common breeds to be used in the analysis.



In [ ]:
# Subset our Data
dog_owner_chars <- subset(dogs2020, select=c("BREED", "BREED_TYPE", "YOB_DOG", "SEX_DOG", "COLOR_DOG", 
                                             "OWNER_ID", "AGE", "SEX", "WEALTH_T_CHF", "INCOME_T_CHF", 
                                             "DISTRICT_NAME"))

# Remove Outlier Breeds (<20 Entries)
dog_owner_chars <- ddply(dog_owner_chars, "BREED", function(d) {if(nrow(d)>19) d else NULL})



# A helper data frame with Unique Breeds
unique_common_breeds <- unique(subset(dog_owner_chars, select=c("BREED")))

### Naive Bayes Model
Using the naiveBayes() function from the e1071 library we create a dataframe with relevant characteristics, where our independent variable is the breed, with our characteristics of owners acting as dependent variables.

In [ ]:
setDT(dog_owner_chars)

dog_owner_chars <- subset(dog_owner_chars, select=c("BREED", "AGE", "SEX"))

# Naive Bayes Implementation
nb <- naiveBayes(BREED ~ ., data=dog_owner_chars, laplace = 0, na.action = na.pass)

Printing the model to the output allows us to view a-priori probabilities as well as conditional probabilities of each breed on Age & Gender respectively.

In [ ]:
nb

A quick glance at the data indicates that of these common breeds, the incidence of a male owner predicts a certain breed with a higher probability in only three cases! (English Bulldog, Rottweiler, Siberian Husky)

It is important to remember our underlying distribution when considering this result. The preponderence of female dog owners in our underlying distribution means that it is very likely that the majority of breeds will be owned by a female owner, and indeed this is what we see. 

### Frequency vs Age
We now use a simple for loop to construct histograms of the conditional probabilities of breed type on age.
While most follow a "normal" distribution (placed in quotes as the distribution of population is ceratainly not completely normal, and there are other aspects of age that determine willingness to own a dog overall, for instance many small children are not able to care for a dog themselves, and similarly for the very old, a dog may be 'too much work'), some do not.

A quick look at the data reveals a couple of highlights, to include that Border Collies seem to skew towards younger owners, while Collies tend towards older owners, while Rottweilers are almost dominated by owners in the 21-30 range.

In [ ]:
# Convert nb into a data frame
nb_df_age <- as.data.frame(nb$tables$AGE)

for (i in 1:length(unique_common_breeds$BREED)) {
  breed_name <- unique_common_breeds$BREED[i]
  breed <- which(nb_df_age$Y == breed_name)
  # create data frame for breed values
  d_age <- data.frame(x = nb_df_age[breed,]$AGE, y = nb_df_age[breed,]$Freq)
  # create plot
  plot <- ggplot(d_age, aes(x = x, y = y, group = 1)) + geom_point() + geom_line() + 
    labs(x = "Age group", y = "Frequency", title = paste(breed_name, "-- Frequency Across Age Groups"))
  print(plot)
}

### Frequency vs Gender
We now repeat the process for the Sex (or Gender) variable, this time utilising a bar graph.
Astute readers will of course notice the predominance of female owners. This is likely due to the overall distribution of dog owners in general. 

In [ ]:
# Similar approach for sex
nb_df_sex <- as.data.frame(nb$tables$SEX)

for (i in 1:length(unique_common_breeds$BREED)) {
  breed_name <- unique_common_breeds$BREED[i]
  breed <- which(nb_df_sex$Y == breed_name)
  # Create data frame for sex values
  # geom_bar() in ggplot2 takes all of the values that one wants to plot in the bar plot and automatically calculates teh frquency. 
  # Here the frequency is already given. 
  # A workaround is to create a data frame that replicates the frequency value with respect the number given for frequency, such that the right amount of values is output
  rep_freq_f <- cbind("x" = rep(nb_df_sex[breed,]$Freq[1], nb_df_sex[breed,]$Freq[1]*100),
                      "y" = rep("f", nb_df_sex[breed,]$Freq[1]*100))
  rep_freq_m <- cbind("x" = rep(nb_df_sex[breed,]$Freq[2], nb_df_sex[breed,]$Freq[2]*100),
                      "y" = rep("m", nb_df_sex[breed,]$Freq[2]*100))
  freq <- as.data.frame(rbind(rep_freq_f, rep_freq_m))
  # Create plot
  plot <- ggplot(freq, aes(x = y)) + geom_bar() + 
    labs(x = "Sex", y = "Frequency", title = paste(breed_name, "-- Frequency Across Genders")) 
  print(plot)
}

---
# Investigation Part 2 : Location, location, location?

## An Analysis in Aggregate

Let's now switch gears. We have performed some categorical analysis on the dataset to predict dog breed selection conditional on characteristics of owners. 

Now we turn our investigation to more general anaylsis of districts. For instance, we may ask ourselves: Does wealth or education influence preponderance of dog ownership? Does the proportion of single family homes vs. apartments play a predictive role? What characteristics of neighbourhoods predict dog ownership, if any? And what sort of policy levers may be used by policy leaders to promote or discourage dog ownership? (If they so choose of course!)

To such matters we now turn our attention. We will begin by investigating the data in aggregate across district. 



### Subset and Aggregate Data
First, we subset the data into those characteristics that are relevant for district level analysis. We then take an aggregate of unique owner ids to produce a figure that represents the total number of dog owners in Zuerich. 

In [ ]:
# First we will need to Subset the Data and Take aggregates to get District Level Statistics
# Relevant Columns w. Duplicates Removed
district_dog <- unique(subset(dogs2020, select=c("OWNER_ID", "DISTRICT_NAME", "WEALTH_T_CHF", "INCOME_T_CHF",
                                                 "BASIC_SCHOOL_PERCENTAGE", "GYMNASIUM_PERCENTAGE", "UNIVERSITY_PERCENTAGE",
                                                 "SINGLE_FAMILY_HOMES", "INFRASTRUCTURE_BUILDINGS", "SMALL_BUILDINGS", "COMMERCIAL_BUILDINGS",
                                                 "APARTMENTS", "FACTORIES_AND_WAREHOUSES", "SPECIAL_ACCOMODATION", "TOTAL_POPULATION",
                                                 "FOREIGN_POPULATION_PERCENTAGE")))

# Aggregation - Count total unique owners and preserve select district data
ddc <- aggregate(OWNER_ID ~ DISTRICT_NAME + WEALTH_T_CHF + INCOME_T_CHF + BASIC_SCHOOL_PERCENTAGE
                 + GYMNASIUM_PERCENTAGE + UNIVERSITY_PERCENTAGE + SINGLE_FAMILY_HOMES
                 + INFRASTRUCTURE_BUILDINGS + SMALL_BUILDINGS + COMMERCIAL_BUILDINGS
                 + APARTMENTS + FACTORIES_AND_WAREHOUSES + SPECIAL_ACCOMODATION + TOTAL_POPULATION + FOREIGN_POPULATION_PERCENTAGE, 
                 data = district_dog, FUN = function(x){NROW(x)})

# Rename Column(s) and Remove Redundant Data
colnames(ddc)[16] <- "TOTAL_UNIQUE_OWNERS"
rm(district_dog)

Now we compute some additional figures for the analysis that may be interesting. These additional attributes per district are described via the following: 

**NON_DOG_OWNERS** = By removing the amount of dog owners from the total population, we can compute a figure for non-dog owners <br>
**PERCENT_DOG_OWNERS** = A simple ratio of dog owners to total population <br>
**TOTAL_BUILDINGS** = We can combine all types of buildings to produce an aggregate <br>
**TOTAL_RESIDENCES** = Similar to TOTAL_BUILDINGS, but for residential type buildings only. This is important to try and distinguish and isolate the effect of a type of building from the overall distribution of total buildings, on dog ownership. <br>

It's important to notice that we have taken a percentage of each home type both over total buildings, as well as total residences. Again, this is to try and isolate the effect of a home type within residences from its frequency in the underlying distribution.

In [ ]:
# Compute Difference
ddc$NON_DOG_OWNERS <- ddc$TOTAL_POPULATION - ddc$TOTAL_UNIQUE_OWNERS
ddc$PERCENT_DOG_OWNERS <- ddc$TOTAL_UNIQUE_OWNERS / ddc$TOTAL_POPULATION

# Compute Total Building Count for Percentages % of Total Buildings and Residential Buildings
ddc$TOTAL_BUILDINGS <- ddc$SINGLE_FAMILY_HOMES + ddc$INFRASTRUCTURE_BUILDINGS + ddc$SMALL_BUILDINGS
                       + ddc$COMMERCIAL_BUILDINGS + ddc$APARTMENTS + ddc$FACTORIES_AND_WAREHOUSES + ddc$SPECIAL_ACCOMODATION
ddc$TOTAL_RESIDENCES <- ddc$SINGLE_FAMILY_HOMES + ddc$APARTMENTS + ddc$SPECIAL_ACCOMODATION

ddc$SINGLE_FAMILY_HOMES_PERCENTAGE <- ddc$SINGLE_FAMILY_HOMES / ddc$TOTAL_BUILDINGS
ddc$SINGLE_FAMILY_HOMES_RESIDENCE_PERCENTAGE <- ddc$SINGLE_FAMILY_HOMES / ddc$TOTAL_RESIDENCES

ddc$APARTMENTS_PERCENTAGE <- ddc$APARTMENTS / ddc$TOTAL_BUILDINGS
ddc$APARTMENTS_RESIDENCE_PERCENTAGE <- ddc$APARTMENTS / ddc$TOTAL_RESIDENCES

ddc$SPECIAL_ACCOMODATION_PERCENTAGE <- ddc$SPECIAL_ACCOMODATION / ddc$TOTAL_BUILDINGS
ddc$SPECIAL_ACCOMODATION_RESIDENCE_PERCENTAGE <- ddc$SPECIAL_ACCOMODATION / ddc$TOTAL_RESIDENCES

Now we display a summary of our table to begin our investigation.

In [ ]:
# Summary Statistics
summary(ddc)

### Linear Model
Let's continue to investigate. First, let's compute scatter plots of dog ownership percentage per district on individual district characteristics, including population, foreign population percentage, wealth & income, and various measures of building composition. 

In [ ]:
# Compute Scatter Plots for Preliminary Investigation of Independent Variables
scatter.smooth(x=ddc$TOTAL_POPULATION, y=ddc$PERCENT_DOG_OWNERS, main="Dog Ownership ~ Population")
scatter.smooth(x=ddc$FOREIGN_POPULATION_PERCENTAGE, y=ddc$PERCENT_DOG_OWNERS, main="Dog Ownership ~ Foreign Population %")

scatter.smooth(x=ddc$WEALTH_T_CHF, y=ddc$PERCENT_DOG_OWNERS, main="Dog Ownership ~ Wealth")
scatter.smooth(x=ddc$INCOME_T_CHF, y=ddc$PERCENT_DOG_OWNERS, main="Dog Ownership ~ Income")

scatter.smooth(x=ddc$SINGLE_FAMILY_HOMES_PERCENTAGE, y=ddc$PERCENT_DOG_OWNERS, main="Dog Ownership ~ Single Fam. Home %")
scatter.smooth(x=ddc$SINGLE_FAMILY_HOMES_RESIDENCE_PERCENTAGE, y=ddc$PERCENT_DOG_OWNERS, main="Dog Ownership ~ Single Fam. Home Res. %")

scatter.smooth(x=ddc$APARTMENTS_PERCENTAGE, y=ddc$PERCENT_DOG_OWNERS, main="Dog Ownership ~ Apartments %")
scatter.smooth(x=ddc$APARTMENTS_RESIDENCE_PERCENTAGE, y=ddc$PERCENT_DOG_OWNERS, main="Dog Ownership ~ Apartments Res. %")

scatter.smooth(x=ddc$SPECIAL_ACCOMODATION_PERCENTAGE, y=ddc$PERCENT_DOG_OWNERS, main="Dog Ownership ~ Spec. Acc. %")
scatter.smooth(x=ddc$SPECIAL_ACCOMODATION_RESIDENCE_PERCENTAGE, y=ddc$PERCENT_DOG_OWNERS, main="Dog Ownership ~ Spec. Acc. Res. %")

Hmm. That's not very satisfying in the majority of the cases. We are seeing some pretty scattered distributions, with questionable correlations for the population and building composition characteristics. We should be very wary of trying to draw any conclusions from those features. 

Fortunately, our wealth and income graphs have produced plots with trends that could be viewed definitively as 'Significant'. How fortunate! Let's continue to investigate this path of our analysis.

First let's compute the correlations for each of these two features with dog ownership percentage.

In [ ]:
# Based upon the plots we decide to investigate the quality of wealth and income on Dog Ownership %
cor_wealth <- cor(ddc$PERCENT_DOG_OWNERS, ddc$WEALTH_T_CHF)
print('Wealth Corr: ')
print(cor_wealth)
cor_income <- cor(ddc$PERCENT_DOG_OWNERS, ddc$INCOME_T_CHF)
print('Income Corr: ')
print(cor_income)

Those are interesting correlations, suggesting that an increase in Wealth or Income is highly correlated with the amount of dog ownership for districts in aggregate. This is a very good foundation for a linear regression. Let's proceed. 

Let's keep in mind some of the fundamental assumptions that allow an ordinary least square (OLS) regression to function as the Best Linear Unbiased Estimator (BLUE), namely that the errors of each y term encompassed in the linear model are uncorrelated with one another, with mean 0 and homoscedastic finite variances. Additionally, there should be strong theoretical/foundational underpinning for a linear relationship.

Let's proceed. 

We compute three regressions, two with a single variable, and one with multivariable input features of both wealth and income.

In [ ]:
# Two Independent Regressions
linearModWealth <- lm(PERCENT_DOG_OWNERS ~ WEALTH_T_CHF, data=ddc)
linearModIncome <- lm(PERCENT_DOG_OWNERS ~ INCOME_T_CHF, data=ddc)
linearModCombi <- lm(PERCENT_DOG_OWNERS ~ WEALTH_T_CHF + INCOME_T_CHF, data=ddc)

Let's compute summaries of each for an overview of our investigation. The computer performs all necessary caculations for us.

In [ ]:
summary(linearModWealth)
summary(linearModIncome)
summary(linearModCombi)

Investigating the data, we are confronted with some facts.

First let us speak to statistical significance. In both of the single-variable models, the predictors are statistically significant, with t-values small enough to trigger the categorical significance code '0', as shown by the outputs. 

With the 'combi' model, the t values explode into insignificant figures. How can this be? We will revisit this shortly. 

Now, in terms of significance of magnitude, it appears both Income and Wealth do indeed have an effect on the percentage of dog ownership in a district. This is great news! However, in terms of magnitude, only Income seems to have an effect that causes us to 'Raise our eyebrows'. 

In plain English: An additional 1000 CHF in average wealth for a district only predicts a 0.0055 % increase (please note this is percent) in dog ownership. It is stastistically significant, but not significant in magnitude. 

However and additional 1000 CHF in average income for a district predicts a 0.0269 % increase in dog ownership. Given the low magnitude of our numbers overall (percentage of dog ownership overall is in the low single digits), we can conclude that this is indeed a reliable result.

So income of a district does indeed predict a higher rate of dog ownership, as stated: each 1000 CHF of additional average income predicts a 0.0269% increase in dog ownership.

To revisit our final question, why then does the combined model break down? If both wealth and income effect dog ownership, shouldn't they both predict dog ownership when combined?

The answer lies in a concept called multi-collinearity. In effect, it is almost certain that income and wealth are correlate <em>with each other</em> as well. When this happens, the variance in dog ownership cannot be reliably discerned from the variance in income and wealth independently, as they are so highly correlated with each other! This can be verified by the F-stastistic for the combined model. What this F statistic says (with a p value of 0.003158, so very significant statistically) is "Does <em>some</em> variable in our idenpendent variables have an effect on our independent variable?" A glance at the value (0.0003158) reveals that indeed, something is affecting the independent variable, dog ownership percentage.

To think analogiously, we know that wealth or income predicts dog ownership, but we don't know which one (or both!), as they almost track each other!


In our case we have made the sensible choice of income to represent both of these variables, which serves as an instrument for the overall depedence of dog ownership on financial condition. The income variable effectively 'encapsulates' the wealth variable in this case. 

Thus we conclude this project and our overall investigation with our most reliable and significant result: Dog Ownership is Conditional on Financial Condition, with Ownership Rates rising with increases in Income and Wealth. 

Thanks for a wonderful course and all the best.<br>
Signed, 

Corey Bothwell<br/>
Nicolleta Farabullini<br/>
Jacob Gelling<br/>
Andris Prokofjevs<br/>
Qasim Warraich<br/>

***